<a href="https://colab.research.google.com/github/Stalin124/TesisScriptdeEntrenamiento/blob/main/Pruebavalidacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ✅ Paso 1: Instalación y Configuración Inicial
!pip install ultralytics roboflow --quiet

from roboflow import Roboflow
from ultralytics import YOLO
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import torch
import shutil

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127

In [2]:
# ✅ Paso 2: Importación de dataset desde Roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="OGHKOE8ss7hxmeOOSa6X")
project = rf.workspace("clasificacin-vehcular").project("precisionaumento1")
version = project.version(2)
dataset = version.download("yolov11")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to PRECISIÓNAUMENTO1-2 in yolov11:: 100%|██████████| 5738/5738 [00:08<00:00, 661.24it/s] 


In [ ]:
# ✅ Paso 3: Aplicación de oversampling a clases minoritarias
# Paso 3.1: Se definen las clases minoritarias
minority_classes = ["BICICLETA", "BUS", "PESADO", "MOTOCICLETA","TAXI"]  # Múltiples clases
n_copias = 5  # Número de veces que se duplicarán las imágenes

# Paso 3.2: Duplicado de imágenes y etiquetas
import os, shutil

images_dir = os.path.join(dataset.location, "train/images")
labels_dir = os.path.join(dataset.location, "train/labels")

for clase in minority_classes:
    for i in range(n_copias):
        for filename in os.listdir(images_dir):
            if clase in filename:
                src_img = os.path.join(images_dir, filename)
                dst_img = os.path.join(images_dir, f"copy{i}_{clase}_{filename}")
                shutil.copy(src_img, dst_img)

                # Verificación de etiquetado correspondiente
                label_file = filename.replace(".jpg", ".txt").replace(".png", ".txt")
                src_lbl = os.path.join(labels_dir, label_file)
                dst_lbl = os.path.join(labels_dir, f"copy{i}_{clase}_{label_file}")
                if os.path.exists(src_lbl):
                    shutil.copy(src_lbl, dst_lbl)

In [3]:
# ✅ Paso 4: Entrenamiento del modelo YOLOv11
model = YOLO("yolo11m.pt")  # Versión de Trabajo yolov11n.pt

results = model.train(
    data=os.path.join(dataset.location, "data.yaml"),
    epochs=50,
    imgsz=960,
    batch=12,
    name="roboflow_yolov11_model",
    device=0 if torch.cuda.is_available() else 'cpu',
    patience=20,
    verbose=True
)

100%|██████████| 38.8M/38.8M [00:00<00:00, 235MB/s]


Ultralytics 8.3.154 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=12, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/PRECISIÓNAUMENTO1-2/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=roboflow_yolov11_model, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=20, perspective=0.0, plots=True

100%|██████████| 755k/755k [00:00<00:00, 105MB/s]

Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  1    111872  ultralytics.nn.modules.block.C3k2            [128, 256, 1, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  1    444928  ultralytics.nn.modules.block.C3k2            [256, 512, 1, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  1   1380352  ultralytics.nn.modules.block.C3k2            [512, 512, 1, True]           
  7                  -1  1   2360320  ultralytics

 13                  -1  1   1642496  ultralytics.nn.modules.block.C3k2            [1024, 512, 1, True]          
 14                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 15             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 16                  -1  1    542720  ultralytics.nn.modules.block.C3k2            [1024, 256, 1, True]          
 17                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
 18            [-1, 13]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 19                  -1  1   1511424  ultralytics.nn.modules.block.C3k2            [768, 512, 1, True]           
 20                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
 21            [-1, 10]  1         0  ultralytics.nn.modules.conv.Concat           [1]  

...

      50/50      13.5G      0.639      0.281     0.8921         49        960: 100%|██████████| 197/197 [03:04<00:00,  1.07it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:06<00:00,  2.16it/s]

                   all        332       2150      0.959      0.901      0.959      0.721



50 epochs completed in 2.742 hours.
Optimizer stripped from runs/detect/roboflow_yolov11_model/weights/last.pt, 40.5MB
Optimizer stripped from runs/detect/roboflow_yolov11_model/weights/best.pt, 40.5MB

Validating runs/detect/roboflow_yolov11_model/weights/best.pt...
Ultralytics 8.3.154 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11m summary (fused): 125 layers, 20,034,658 parameters, 0 gradients, 67.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:09<00:00,  1.41it/s]


                   all        332       2150       0.96      0.901      0.959      0.721
             BICICLETA         17         18      0.966      0.833      0.935      0.645
                   BUS         35         35      0.969      0.943      0.971      0.737
           MOTOCICLETA        141        188      0.942      0.809      0.936      0.598
            PARTICULAR        319       1228      0.961      0.952       0.98      0.802
                PESADO         66         72      0.954      0.889      0.939      0.743
                  TAXI        293        609      0.972      0.981       0.99        0.8
Speed: 0.4ms preprocess, 13.4ms inference, 0.0ms loss, 4.2ms postprocess per image
Results saved to runs/detect/roboflow_yolov11_model
